# Demonstration of functions operating on UV spectra for NurF

In [ ]:
from ess.loki.nurf import utils, uv, plot
from ess.loki.nurf import ill_auxilliary_funcs as ill 

# standard library imports
import itertools
import os
from typing import Optional, Type, Union

# related third party imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.gridspec as gridspec
from IPython.display import display, HTML
from scipy.optimize import leastsq  # needed for fitting of turbidity

# local application imports
import scippnexus as snx
import scipp as sc
from scipp.ndimage import median_filter

In [ ]:
# Prepare for export to .dat for uv and fluo

# path to LOKI-like files
process_folder='/Users/gudlo523/Library/CloudStorage/OneDrive-LundUniversity/UU/ILL_947_June_2021/212/d22/exp_9-13-947/processed/ess_version'
# change to folder
os.chdir(process_folder)

# export path for .dat files
path_output='/Users/gudlo523/Library/CloudStorage/OneDrive-LundUniversity/UU/ILL_947_June_2021/212/d22/exp_9-13-947/processed/ess_version/dat-files'

# experimental data sets
exp5= [66017, 66020, 66023, 66026, 66029, 66032, 66034, 66037, 66040, 66043, 66046]
exp6= [66050, 66053, 66056, 66059, 66062, 66065, 66068, 66071, 66074, 66077, 66080]

exp2= [65925, 65927, 65930, 65933, 65936, 65939, 65942, 65945, 65948, 65951, 65954, 65957]
exp3= [65962, 65965, 65968, 65971, 65974, 65977, 65980, 65983, 65986, 65989, 65992]

exp7= [66083, 66086, 66089, 66092, 66095, 66098, 66101, 66104, 66107, 66110, 66113]
exp8= [66116, 66119, 66122, 66125, 66128, 66131, 66134, 66137, 66140, 66143, 66146]

In [ ]:
name='066017.nxs'
exp_meth='uv'

# load a file and caluclate corrected uv spectra
uv_dict=utils.load_nurfloki_file(name,exp_meth)
uv_da=uv.normalize_uv(**uv_dict)

display(uv_da)

#quick plot
display(sc.plot(sc.collapse(uv_da, keep="wavelength")))

In [ ]:
# extract value for given wavelength
wavelength=sc.scalar(280,unit='nm')
tol= sc.scalar(0.5,unit='nm')
res=uv.uv_peak_int(uv_da, wavelength, tol=None)
#returns a dict, #TODO: do we want a Dataset here? 
print(res)

How to average mutliple UV spectra in one Loki.nxs file?

In [ ]:
name='066017.nxs'
#How to average multiple UV spectra in one LoKI.nxs file?

out1=uv.load_and_normalize_uv(name)
display(out1)

# Apply uv.average_uv function to Loki.nxs
out2=uv.average_uv(name)
display(out2)



Cell below shows how to perform a turbidity correction on data from a single input file. If the user wishes to apply a median_filter beforehand, the user can apply this manually by adding one line of code, see next cell.

In [ ]:
name='066017.nxs'

#How to perform a turbiity correction on one input file?
#uv_dict=utils.load_nurfloki_file(name,exp_meth)
#uv_da=uv.normalize_uv(**uv_dict)

# Or load like this:
uv_da=uv.load_and_normalize_uv(name)

uv_turb_corr=uv.uv_turbidity_fit(uv_da, fit_llim=None, fit_ulim=None, b_llim=None, b_ulim=None, m=None)
display(uv_turb_corr)



#How to plot the turbidity correction for visual inspection? 
plot.plot_uv_turbidity_fit(uv_da, fit_llim=None, fit_ulim=None, b_llim=None, b_ulim=None, m=None)


Cell below shows how to perform a turbidity correction on data from a single input file. Here is the example where a median filter is applied before hand.

In [ ]:
name='066017.nxs'

# Or load like this:
uv_da=uv.load_and_normalize_uv(name)

# How to apply a median filter?
kernel_size=10
#uv_da=utils.nurf_median_filter(uv_da, kernel_size=kernel_size)

#uv_turb_corr_mfilt=uv.uv_turbidity_fit(uv_da, fit_llim=None, fit_ulim=None, b_llim=None, b_ulim=None, m=None)
#display(uv_turb_corr_mfilt)

# Median filter in one line with turbdity_fit
uv_turb_corr_mfilt=uv.uv_turbidity_fit(utils.nurf_median_filter(uv_da, kernel_size=kernel_size), fit_llim=None, fit_ulim=None, b_llim=None, b_ulim=None, m=None)


#How to plot the turbidity correction for visual inspection? 
plot.plot_uv_turbidity_fit(uv_turb_corr_mfilt, fit_llim=None, fit_ulim=None, b_llim=None, b_ulim=None, m=None)


Cell below shows how to perform a turbidity fit on multiple input files **without** previous applying a median filter

In [ ]:

scanlist = [66017, 66020]
filelist=ill.complete_fname(scanlist)

# apply turbidity correction to multiple files
# create first a dict of sc.DataArray, where each array corresponds to a Loki.file
dict_da={name:uv.load_and_normalize_uv(name) for name in [name for name in filelist]} 
# alternative that works, but maybe not so nice :-)
#d={name:uv.load_and_normalize_uv(name) for name in [name for name in ill.complete_fname(scanlist)]} 


# apply turbidity fit to all spectra inside each Loki.file and to multiple files at once
res_da=uv.uv_multi_turbidity_fit(dict_da)

display(res_da)

# plot turbidity corrected files
plot.plot_uv_multi_turbidity_fit(dict_da)

Cell below shows how to perform a turbidity fit on multiple input files **with** applying a median filter

In [ ]:
scanlist = [66017, 66020]
filelist=ill.complete_fname(scanlist)

kernel_size=15

# apply turbidity correction to multiple files
# create first a dict of sc.DataArray, where each array corresponds to a Loki.file
# for each sc.DataArray the nurf_median_filter is applied with above kernel_size
dict_da={name:utils.nurf_median_filter(uv.load_and_normalize_uv(name), kernel_size=kernel_size) for name in [name for name in filelist]} 


# apply turbidity fit to all spectra inside each Loki.file and to multiple files at once
res_da=uv.uv_multi_turbidity_fit(dict_da)

display(res_da)

# plot turbidity corrected files
plot.plot_uv_multi_turbidity_fit(dict_da)

How to gather the spectra of multiple files in one sc.DataArray or in one sc.Dataset?


In [ ]:
#How to gather the spectra of multiple files in one sc.DataArray or in one sc.Dataset?
filesetlist=ill.complete_fname(exp5)
uv_da=uv.gather_uv_set(filesetlist)
display(uv_da)

#if all files contain the same number of UV spectra, a sc.DataSet is returned 
filesetlist_3files=ill.complete_fname([66017, 66020, 66023])
uv_ds=uv.gather_uv_set(filesetlist_3files)
display(uv_ds)


In [ ]:
%%time

name='066017.nxs'
da = uv.load_and_normalize_uv(name)

#How to apply the scipp.ndimage.median_filter to an sc.DataArray? 
#Current NUrF has non-equaled spaced data, only int is accepeted.
#Otherwise: kernel_size=sc.scalar(2.5, units='nm') as example should work.
kernel_size=15 
da_filt=utils.nurf_median_filter(da, kernel_size=kernel_size)

#plot all contributions
legend_props = {"show": True, "loc": 1}
fig1=sc.plot(sc.collapse(da, keep='wavelength'),title='before any median filter',legend=legend_props, marker='.')
fig2=sc.plot(sc.collapse(da_filt, keep='wavelength'), marker='.', title=f'after median filter - scipp, size={kernel_size} ',legend=legend_props)

display(fig1,fig2)


